In [ ]:
import pypsa
import pandas as pd
import xarray as xr

n = pypsa.Network("marginal_system_value/results/2050-rd2-200000-24h/optimized_2050_rd2_200000_network.nc")

print(n)

In [ ]:
carriers_to_modify_SBSP = ["SBSP"]

sbsp_mask = n.generators['carrier'].isin(carriers_to_modify_SBSP)

n.generators.loc[sbsp_mask, 'p_nom_extendable'] = False

scale_factor = 200 / 291.01
n.generators.loc[sbsp_mask, 'p_nom'] = n.generators.loc[sbsp_mask, 'p_nom_opt'] * scale_factor

In [ ]:
snapshots_to_optimize = n.snapshots[:366]
n.optimize(snapshots=snapshots_to_optimize, solver_name='gurobi')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

supply_data = n.statistics.supply(comps=["Generator", "StorageUnit", "Store"], aggregate_time=False)

supply_data = supply_data.droplevel(0).T

print("Available columns (energy types):")
print(supply_data.columns)

supply_data.to_csv('marginal_system_value/results/2050-rd2-200000-24h_200/2050_middle_rd2_200000_hourly_energy_supply.csv', index=True)

In [ ]:
# CapEx
total_capex = n.statistics.capex()

# OpEx
total_opex = n.statistics.opex(aggregate_time="sum")

# total_cost
total_cost = total_capex + total_opex

with open("marginal_system_value/results/2050-rd2-200000-24h_200/total_cost_summary_rd2_200000.txt", "w") as f:
    f.write(f"Total CapEx: {total_capex}\n")
    f.write(f"Total OpEx: {total_opex}\n")
    f.write(f"Total Cost (CapEx + OpEx): {total_cost}\n")

print("Results saved to total_cost_summary.txt")